In [1]:
import pandas as pd
import ast
import random
import time
import copy

In [2]:
random.seed(16)
# random.seed(17)
# random.seed(18)

## Import data

In [3]:
job_shop_df = pd.read_csv('job_shop_data.csv', sep=';', index_col=False)
job_shop_df['problem_instance'] = job_shop_df['problem_instance'].apply(lambda x: ast.literal_eval(x))

In [4]:
job_shop_df.head()

,instance_id,num_jobs,num_machines,problem_instance
0,5j5m,5,5,"[[(3, 6), (1, 4), (2, 1), (0, 3), (4, 6)], [(0..."
1,5j6m,5,6,"[[(5, 9), (3, 5), (0, 7), (2, 2), (1, 2), (4, ..."
2,5j7m,5,7,"[[(5, 7), (6, 3), (3, 4), (1, 1), (2, 4), (4, ..."
3,5j8m,5,8,"[[(1, 7), (3, 9), (6, 3), (0, 8), (2, 1), (7, ..."
4,5j9m,5,9,"[[(7, 6), (4, 10), (8, 1), (2, 2), (1, 10), (5..."


## Functions to build up the HillClimber and FFA-HillClimber algorithms

In [5]:
def random_solution_permutation_representation(problem_instance):
    num_jobs = len(problem_instance)
    num_machines = len(problem_instance[0])
    solution = [job for job in range(num_jobs) for i in range(num_machines)]
    random.shuffle(solution)
    return solution


def permutation_to_makespan(permutation_rep, problem_instance):
    makespan = 0
    current_time = 0
    num_jobs = len(problem_instance)
    num_machines = len(problem_instance[0])
    
    # keeps track of the number of times that a job has already occured
    job_count = [0 for job in range(num_jobs)]
    
    # keep track of the current end time of each job
    jobs_end_time = [0 for job in range(num_jobs)]
    
    # keep track of the current end time of each machine
    machines_end_time = [0 for machine in range(num_machines)]
    
    for job in permutation_rep:

        job_list = problem_instance[job] #gives the list of a job in which we find the sequence of machines and the processing times on those machines
        job_index = job_count[job] #to find the index at which the relevant machine number and processing time info is stored

        machine = job_list[job_index][0] #the machine that this job should be processed on now
        processing_time = job_list[job_index][1] #the relevant processing time
        
        
        # increase job occurrence for next iterations
        job_count[job] += 1
        
        # the job can start on this machine when both this job and this machine have finished their last run
        job_machine_end_time = max(jobs_end_time[job], machines_end_time[machine])
        new_end_time = job_machine_end_time + processing_time #add current processing time to the previous end time
        
        # assign new end time to both the relevant job and the relevant machine
        jobs_end_time[job] = new_end_time
        machines_end_time[machine] = new_end_time
        
    # when all jobs in the permutation representation have been assigned to the machines with their relevant processing times, the total makespan of the problem instance should be the max end time of either / both the job end times and the machine end times
    makespan = max(jobs_end_time)

    return makespan


def swap_jobs(permutation_rep):
    swap_list=permutation_rep
    random_index_1 = random.randint(0, len(swap_list)-1)
    random_index_2 = random.randint(0, len(swap_list)-1)
    
    # only perform swap if two different job IDs will be swapped
    while swap_list[random_index_1] == swap_list[random_index_2]:
        random_index_2 = random.randint(0, len(swap_list)-1)
        
    swap_candidate_1 = swap_list[random_index_1]
    swap_candidate_2 = swap_list[random_index_2]
    swap_list[random_index_1] = swap_candidate_2
    swap_list[random_index_2] = swap_candidate_1

    return swap_list

## HillClimber

In [34]:
def hillClimber(problem_instance, num_evaluations):
    # create current solution perm rep and compute its makespan
    current_solution = random_solution_permutation_representation(problem_instance)
    current_makespan = permutation_to_makespan(current_solution, problem_instance)
    
    # to visualize convergence in the analysis, all best so far solutions need to be saved
    all_best_so_far_makespans = []
    
    # start hillClimber mutations and evaluations
    for evaluation in range(num_evaluations):
        all_best_so_far_makespans.append(current_makespan)
        
        # copy solution first so the current solution will not get altered by the mutation
        new_solution = copy.deepcopy(current_solution)
        new_solution = swap_jobs(new_solution)
        new_makespan = permutation_to_makespan(new_solution, problem_instance)
#         print(current_solution)
#         print(new_solution)
#         print('current makespan: {current_makespan}, new makespan: {new_makespan}'.format(current_makespan=current_makespan, new_makespan=new_makespan))
        if new_makespan <= current_makespan:
            current_solution = new_solution
            current_makespan = new_makespan
        
    return current_solution, current_makespan, all_best_so_far_makespans

### Run HC with 1,000,000 evaluations per problem instance

In [47]:
results = []

EVALUATIONS = 1000000
start_total_time = time.time()
instance_id = 0
for problem_instance in job_shop_df['problem_instance']:
    start_eval_time = time.time()
    evaluation_result = hillClimber(problem_instance, EVALUATIONS)
    results.append((instance_id, evaluation_result[0], evaluation_result[1], evaluation_result[2]))
    instance_id += 1
    print ("Instance " + str(instance_id) + " evaluation time: %s seconds" % (time.time() - start_eval_time))
    
print("Total time: %s seconds" % (time.time() - start_total_time))

Instance 1 evaluation time: 28.667470932006836 seconds
Instance 2 evaluation time: 32.078670024871826 seconds
Instance 3 evaluation time: 35.79343914985657 seconds
Instance 4 evaluation time: 41.723456144332886 seconds
Instance 5 evaluation time: 43.973109006881714 seconds
Instance 6 evaluation time: 48.086544036865234 seconds
Instance 7 evaluation time: 51.8151741027832 seconds
Instance 8 evaluation time: 56.10477304458618 seconds
Instance 9 evaluation time: 61.208348989486694 seconds
Instance 10 evaluation time: 64.92171502113342 seconds
Instance 11 evaluation time: 71.34520483016968 seconds
Instance 12 evaluation time: 76.29235291481018 seconds
Instance 13 evaluation time: 77.58966398239136 seconds
Instance 14 evaluation time: 81.83807182312012 seconds
Instance 15 evaluation time: 85.83312320709229 seconds
Instance 16 evaluation time: 90.19664072990417 seconds
Instance 17 evaluation time: 96.15975499153137 seconds
Instance 18 evaluation time: 98.91682314872742 seconds
Instance 19 ev

Instance 149 evaluation time: 210.353905916214 seconds
Instance 150 evaluation time: 242.94195079803467 seconds
Instance 151 evaluation time: 277.4563488960266 seconds
Instance 152 evaluation time: 309.9760720729828 seconds
Instance 153 evaluation time: 345.465106010437 seconds
Instance 154 evaluation time: 380.71402502059937 seconds
Instance 155 evaluation time: 411.341511964798 seconds
Instance 156 evaluation time: 442.9917299747467 seconds
Instance 157 evaluation time: 479.73620104789734 seconds
Instance 158 evaluation time: 513.3240478038788 seconds
Instance 159 evaluation time: 544.330892086029 seconds
Instance 160 evaluation time: 580.090460062027 seconds
Instance 161 evaluation time: 614.8276271820068 seconds
Instance 162 evaluation time: 648.4997158050537 seconds
Instance 163 evaluation time: 682.2811958789825 seconds
Instance 164 evaluation time: 720.3890118598938 seconds
Instance 165 evaluation time: 751.5690996646881 seconds
Instance 166 evaluation time: 781.3426802158356 se

Instance 295 evaluation time: 332.24220275878906 seconds
Instance 296 evaluation time: 396.3735728263855 seconds
Instance 297 evaluation time: 460.8170599937439 seconds
Instance 298 evaluation time: 524.6414589881897 seconds
Instance 299 evaluation time: 585.5557861328125 seconds
Instance 300 evaluation time: 649.1930258274078 seconds
Instance 301 evaluation time: 715.7587940692902 seconds
Instance 302 evaluation time: 769.8906760215759 seconds
Instance 303 evaluation time: 833.1748208999634 seconds
Instance 304 evaluation time: 897.5582880973816 seconds
Instance 305 evaluation time: 959.3153820037842 seconds
Instance 306 evaluation time: 1026.8139550685883 seconds
Instance 307 evaluation time: 1092.6183841228485 seconds
Instance 308 evaluation time: 1153.3359739780426 seconds
Instance 309 evaluation time: 1216.6513390541077 seconds
Instance 310 evaluation time: 1277.217963218689 seconds
Instance 311 evaluation time: 1340.3162109851837 seconds
Instance 312 evaluation time: 1406.2355599

## Save results

In [ ]:
results_df = pd.DataFrame(results, columns = ('instance_id', 'best_solution', 'best_makespan', 'convergence'))
results_df.head()

In [ ]:
results_df.to_csv('hc_1000000_evals_cor.csv', sep=';', index=False)
# results_df.to_csv('hc_1000000_evals_cor2.csv', sep=';', index=False)
# results_df.to_csv('hc_1000000_evals_cor3.csv', sep=';', index=False)

## FFA HillClimber

In [6]:
def FFAhillClimber(problem_instance, num_evaluations):
    # initiate fitness dictionary
    fitness_dict = {}
    
    # create current solution perm rep and compute its makespan
    current_solution = random_solution_permutation_representation(problem_instance)
    current_makespan = permutation_to_makespan(current_solution, problem_instance)
    
    # keep track of the best overall solution found
    best_overall_makespan = current_makespan
    best_overall_solution = current_solution
    
    # to visualize convergence in the analysis, all best so far solutions need to be saved
    all_best_so_far_makespans = []
    all_current_makespans = []
    
    # set fitness frequency of found makespan to 1
    fitness_dict[current_makespan] = 1
    
    # start hillClimber mutations and evaluations
    for evaluation in range(num_evaluations):
        all_best_so_far_makespans.append(best_overall_makespan)
        all_current_makespans.append(current_makespan)
        
        # increase fitness frequency of current makespan
        fitness_dict[current_makespan] += 1
        
        # copy solution first so the current solution will not get altered by the mutation
        new_solution = copy.deepcopy(current_solution)
        new_solution = swap_jobs(new_solution)
        new_makespan = permutation_to_makespan(new_solution, problem_instance)

        # increase fitness frequency of new makespan
        if new_makespan in fitness_dict:
            fitness_dict[new_makespan] += 1
        else:
            fitness_dict[new_makespan] = 1
        
#         print(current_solution)
#         print(new_solution)
#         print('current makespan: {current_makespan}, new makespan: {new_makespan}'.format(current_makespan=current_makespan, new_makespan=new_makespan))
        if fitness_dict[new_makespan] <= fitness_dict[current_makespan]:
            current_solution = new_solution
            current_makespan = new_makespan
            
            if current_makespan <= best_overall_makespan:
                best_overall_solution = current_solution
                best_overall_makespan = current_makespan
        
    return current_solution, current_makespan, best_overall_solution, best_overall_makespan, all_best_so_far_makespans, all_current_makespans

### Run FFA-HC with 1,000,000 evaluations per problem instance

In [7]:
FFA_results = []

EVALUATIONS = 1000000
start_total_time = time.time()
instance_id = 0
for problem_instance in job_shop_df['problem_instance']:
    start_eval_time = time.time()
    evaluation_result = FFAhillClimber(problem_instance, EVALUATIONS)
    FFA_results.append((instance_id, evaluation_result[0], evaluation_result[1], evaluation_result[2], evaluation_result[3], evaluation_result[4], evaluation_result[5]))
    instance_id += 1
    print ("Instance " + str(instance_id) + " evaluation time: %s seconds" % (time.time() - start_eval_time))
    
print("Total time: %s seconds" % (time.time() - start_total_time))

Instance 1 evaluation time: 28.4397189617157 seconds
Instance 2 evaluation time: 33.68164801597595 seconds
Instance 3 evaluation time: 36.41832399368286 seconds
Instance 4 evaluation time: 39.412240982055664 seconds
Instance 5 evaluation time: 44.100927114486694 seconds
Instance 6 evaluation time: 47.824944734573364 seconds
Instance 7 evaluation time: 52.46005582809448 seconds
Instance 8 evaluation time: 56.177855014801025 seconds
Instance 9 evaluation time: 61.14599895477295 seconds
Instance 10 evaluation time: 65.19693684577942 seconds
Instance 11 evaluation time: 69.05049920082092 seconds
Instance 12 evaluation time: 73.67844295501709 seconds
Instance 13 evaluation time: 77.88202214241028 seconds
Instance 14 evaluation time: 81.9320273399353 seconds
Instance 15 evaluation time: 86.40462064743042 seconds
Instance 16 evaluation time: 90.4984359741211 seconds
Instance 17 evaluation time: 98.80319213867188 seconds
Instance 18 evaluation time: 106.13843202590942 seconds
Instance 19 evalu

Instance 149 evaluation time: 211.51033115386963 seconds
Instance 150 evaluation time: 244.1915090084076 seconds
Instance 151 evaluation time: 278.52445101737976 seconds
Instance 152 evaluation time: 313.14120602607727 seconds
Instance 153 evaluation time: 344.84049582481384 seconds
Instance 154 evaluation time: 379.8366310596466 seconds
Instance 155 evaluation time: 411.7049160003662 seconds
Instance 156 evaluation time: 447.0713210105896 seconds
Instance 157 evaluation time: 481.2746801376343 seconds
Instance 158 evaluation time: 514.162563085556 seconds
Instance 159 evaluation time: 547.4961409568787 seconds
Instance 160 evaluation time: 579.2842886447906 seconds
Instance 161 evaluation time: 620.6183869838715 seconds
Instance 162 evaluation time: 648.0384957790375 seconds
Instance 163 evaluation time: 707.2258241176605 seconds
Instance 164 evaluation time: 762.1916108131409 seconds
Instance 165 evaluation time: 762.9034512042999 seconds
Instance 166 evaluation time: 785.95571398735

Instance 295 evaluation time: 339.6961588859558 seconds
Instance 296 evaluation time: 404.36479091644287 seconds
Instance 297 evaluation time: 470.7496621608734 seconds
Instance 298 evaluation time: 534.5097851753235 seconds
Instance 299 evaluation time: 600.657721042633 seconds
Instance 300 evaluation time: 674.9421586990356 seconds
Instance 301 evaluation time: 732.1146428585052 seconds
Instance 302 evaluation time: 799.556009054184 seconds
Instance 303 evaluation time: 863.1832098960876 seconds
Instance 304 evaluation time: 928.7691702842712 seconds
Instance 305 evaluation time: 994.0136041641235 seconds
Instance 306 evaluation time: 1062.8472909927368 seconds
Instance 307 evaluation time: 1122.0649108886719 seconds
Instance 308 evaluation time: 1194.0182092189789 seconds
Instance 309 evaluation time: 1259.1543970108032 seconds
Instance 310 evaluation time: 1318.2558319568634 seconds
Instance 311 evaluation time: 1389.4172492027283 seconds
Instance 312 evaluation time: 1461.65788698

## Save results

In [8]:
FFAresults_df = pd.DataFrame(FFA_results, columns = ('instance_id', 'least_frequent_solution', 'least_frequent_makespan', 'best_solution', 'best_makespan', 'convergence', 'search_progression'))
FFAresults_df.head()

,instance_id,least_frequent_solution,least_frequent_makespan,best_solution,best_makespan,convergence,search_progression
0,0,"[0, 4, 4, 0, 3, 2, 2, 1, 1, 0, 0, 4, 4, 3, 3, ...",113,"[2, 3, 1, 1, 1, 0, 0, 0, 4, 4, 2, 1, 3, 2, 4, ...",48,"[65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 6...","[65, 65, 65, 74, 77, 83, 83, 83, 79, 71, 71, 6..."
1,1,"[1, 1, 2, 2, 4, 0, 3, 3, 4, 3, 0, 3, 3, 2, 1, ...",52,"[3, 1, 2, 0, 2, 4, 1, 4, 2, 3, 1, 3, 2, 0, 4, ...",48,"[61, 61, 61, 61, 60, 60, 60, 60, 60, 60, 60, 6...","[61, 61, 67, 62, 60, 60, 60, 60, 63, 63, 63, 6..."
2,2,"[0, 0, 3, 1, 1, 4, 4, 4, 1, 1, 0, 4, 4, 4, 4, ...",187,"[1, 0, 4, 1, 1, 2, 0, 3, 4, 2, 0, 3, 3, 0, 2, ...",52,"[73, 64, 64, 64, 58, 58, 58, 58, 58, 58, 58, 5...","[73, 64, 70, 70, 58, 58, 67, 67, 69, 69, 69, 6..."
3,3,"[1, 1, 1, 2, 1, 1, 2, 1, 2, 4, 4, 2, 4, 1, 2, ...",144,"[1, 2, 3, 2, 0, 4, 2, 4, 4, 1, 2, 3, 1, 1, 3, ...",54,"[83, 71, 71, 71, 71, 71, 71, 71, 71, 71, 71, 7...","[83, 71, 88, 82, 80, 80, 82, 82, 82, 82, 83, 8..."
4,4,"[3, 3, 0, 3, 0, 0, 0, 2, 0, 4, 0, 3, 0, 0, 1, ...",219,"[0, 3, 1, 1, 1, 0, 2, 3, 0, 2, 2, 1, 2, 0, 4, ...",61,"[97, 82, 82, 82, 82, 82, 82, 82, 82, 82, 82, 8...","[97, 82, 82, 119, 119, 129, 122, 122, 129, 112..."


In [9]:
FFAresults_df.tail()

,instance_id,least_frequent_solution,least_frequent_makespan,best_solution,best_makespan,convergence,search_progression
415,415,"[17, 46, 96, 64, 70, 16, 86, 47, 43, 99, 1, 30...",2088,"[75, 36, 89, 68, 14, 31, 35, 61, 68, 65, 29, 9...",763,"[1133, 1133, 1133, 1133, 1133, 1133, 1133, 113...","[1133, 1133, 1140, 1151, 1165, 1164, 1164, 116..."
416,416,"[48, 16, 58, 6, 74, 44, 77, 58, 57, 46, 40, 89...",1829,"[38, 15, 5, 73, 38, 61, 56, 72, 29, 62, 90, 18...",751,"[1141, 1140, 1140, 1140, 1140, 1140, 1140, 114...","[1141, 1140, 1140, 1140, 1146, 1159, 1162, 116..."
417,417,"[15, 6, 66, 14, 4, 27, 59, 19, 69, 4, 61, 81, ...",1573,"[97, 37, 25, 86, 92, 16, 43, 78, 15, 91, 60, 0...",750,"[1114, 1109, 1107, 1107, 1107, 1107, 1107, 110...","[1114, 1109, 1107, 1107, 1108, 1112, 1112, 111..."
418,418,"[38, 64, 82, 7, 23, 21, 95, 36, 97, 23, 70, 99...",1839,"[44, 70, 10, 71, 51, 74, 84, 89, 31, 18, 22, 9...",781,"[1187, 1181, 1181, 1181, 1181, 1181, 1181, 118...","[1187, 1181, 1195, 1195, 1195, 1209, 1188, 118..."
419,419,"[66, 29, 6, 19, 80, 60, 8, 92, 14, 15, 22, 27,...",2033,"[10, 99, 37, 71, 55, 87, 55, 50, 68, 27, 39, 6...",764,"[1160, 1160, 1151, 1140, 1140, 1140, 1140, 114...","[1160, 1161, 1151, 1140, 1150, 1158, 1164, 115..."


In [10]:
FFAresults_df.to_csv('ffahc_1000000_evals_cor.csv', sep=';', index=False)
# FFAresults_df.to_csv('ffahc_1000000_evals_cor2.csv', sep=';', index=False)
# FFAresults_df.to_csv('ffahc_1000000_evals_cor3.csv', sep=';', index=False)

## Trying some plots

In [12]:
import numpy as np
import matplotlib.pyplot as plt

In [57]:
hc_convergence = results_df['convergence']
hc_smallest_instance_conv = hc_convergence[0]

ffahc_convergence = FFAresults_df['convergence']
ffahc_smallest_instance_conv = ffahc_convergence[0]

In [58]:
hc_smallest_instance_conv

[60,
 60,
 60,
 60,
 60,
 59,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,


In [59]:
ffahc_smallest_instance_conv

[56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 56,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 54,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 52,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 50,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,
 48,


In [ ]:
iterations = range(1, len(hc_smallest_instance_conv) + 1)

In [11]:
plt.figure(figsize=(8, 6))

# plt.plot(iterations, hc_smallest_instance_conv, label='HC')
plt.plot(iterations, ffahc_smallest_instance_conv, label='FFA-HC')

plt.xlabel('Evaluations')
plt.ylabel('Objective value')
plt.title('Convergence Comparison of HC and FFA-HC')
plt.legend()

plt.xscale('log')
plt.grid(True)

plt.show()


KeyboardInterrupt



In [62]:
hc_convergence = results_df['convergence']
hc_largest_instance_conv = hc_convergence[419]

ffahc_convergence = FFAresults_df['convergence']
ffahc_largest_instance_conv = ffahc_convergence[419]

In [ ]:
evaluations = range(1, len(hc_smallest_instance_conv) + 1)
plt.figure(figsize=(8, 6))

plt.plot(evaluations, hc_largest_instance_conv, label='HC')
plt.plot(evaluations, ffahc_largest_instance_conv, label='FFA-HC')

plt.xlabel('Evaluations')
plt.ylabel('Objective value')
plt.title('Convergence Comparison of HC and FFA-HC')
plt.legend()

plt.xscale('log')
plt.grid(True)

plt.show()

### Where did HC outperform FFA-HC?

In [64]:
hc_results_winners = results_df[results_df['best_makespan'] < FFAresults_df['best_makespan']]

In [65]:
len(hc_results_winners)

321

In [66]:
hc_results_winners

,instance_id,best_solution,best_makespan,convergence
23,23,"[8, 3, 1, 3, 9, 5, 0, 8, 8, 5, 3, 7, 5, 1, 9, ...",76,"[145, 145, 132, 132, 132, 132, 129, 129, 129, ..."
25,25,"[4, 1, 8, 2, 8, 1, 5, 7, 0, 8, 0, 2, 8, 9, 1, ...",89,"[137, 137, 137, 137, 137, 137, 137, 137, 137, ..."
27,27,"[1, 6, 3, 5, 6, 3, 6, 9, 8, 3, 1, 8, 0, 1, 3, ...",110,"[222, 215, 215, 215, 215, 208, 208, 207, 204, ..."
28,28,"[7, 3, 4, 4, 9, 6, 1, 2, 5, 7, 2, 0, 1, 7, 9, ...",101,"[182, 182, 182, 179, 179, 169, 169, 169, 169, ..."
29,29,"[8, 4, 0, 1, 4, 8, 7, 9, 3, 1, 6, 6, 5, 0, 2, ...",110,"[207, 201, 201, 201, 198, 189, 189, 189, 189, ..."
...,...,...,...,...
415,415,"[90, 5, 84, 0, 37, 66, 90, 14, 29, 82, 15, 29,...",634,"[1120, 1120, 1120, 1111, 1111, 1106, 1102, 110..."
416,416,"[33, 19, 24, 43, 72, 18, 40, 96, 81, 91, 82, 5...",616,"[1094, 1089, 1076, 1076, 1076, 1076, 1076, 107..."
417,417,"[33, 68, 61, 70, 15, 49, 64, 66, 86, 97, 78, 5...",613,"[1098, 1096, 1096, 1089, 1089, 1087, 1087, 108..."
418,418,"[53, 9, 47, 28, 61, 17, 24, 66, 36, 91, 32, 49...",633,"[1210, 1210, 1210, 1210, 1210, 1210, 1169, 116..."


### Where did FFA-HC outperform HC?

In [67]:
ffahc_results_winners = FFAresults_df[FFAresults_df['best_makespan'] < results_df['best_makespan']]

In [68]:
len(ffahc_results_winners)

4

In [69]:
ffahc_results_winners

,instance_id,least_frequent_solution,least_frequent_makespan,best_solution,best_makespan,convergence,search_progression
8,8,"[2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 0, 1, 0, 3, 3, ...",327,"[4, 0, 2, 4, 1, 4, 2, 0, 4, 3, 1, 4, 1, 2, 2, ...",97,"[149, 144, 144, 144, 139, 131, 131, 131, 131, ...","[149, 144, 162, 162, 139, 131, 137, 137, 137, ..."
24,24,"[0, 6, 7, 3, 8, 5, 8, 0, 2, 1, 2, 7, 8, 2, 1, ...",127,"[7, 8, 4, 5, 4, 0, 9, 5, 6, 1, 4, 0, 6, 8, 9, ...",76,"[120, 120, 120, 120, 120, 120, 120, 119, 119, ...","[120, 125, 124, 124, 124, 136, 136, 119, 119, ..."
26,26,"[9, 3, 0, 3, 5, 9, 9, 0, 3, 5, 8, 0, 3, 1, 8, ...",357,"[3, 0, 6, 8, 8, 7, 7, 2, 5, 4, 1, 3, 2, 6, 4, ...",99,"[174, 174, 174, 174, 174, 174, 174, 174, 174, ...","[174, 180, 198, 200, 200, 205, 195, 197, 190, ..."
31,31,"[2, 2, 4, 0, 7, 6, 7, 7, 7, 6, 6, 1, 9, 9, 5, ...",318,"[5, 2, 3, 2, 8, 4, 3, 5, 4, 2, 6, 2, 4, 6, 1, ...",114,"[196, 196, 187, 187, 187, 187, 187, 187, 187, ...","[196, 196, 187, 189, 189, 193, 193, 191, 191, ..."
